# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 13 2022 10:35AM,239450,15,VT80173613,"Virtus Pharmaceuticals, LLC",Released
1,Jun 13 2022 10:34AM,239448,15,8138380,"Carwin Pharmaceutical Associates, LLC",Released
2,Jun 13 2022 10:34AM,239447,15,8126955,"Mizner Bioscience, LLC",Released
3,Jun 13 2022 10:34AM,239447,15,8126956,"Mizner Bioscience, LLC",Released
4,Jun 13 2022 10:34AM,239447,15,8126957,"Mizner Bioscience, LLC",Released
5,Jun 13 2022 10:30AM,239445,10,8139025,Beach Products Inc,Released
6,Jun 13 2022 10:29AM,239442,15,PNC356392A,"Person & Covey, Inc.",Released
7,Jun 13 2022 10:29AM,239442,15,PNC356609A,"Person & Covey, Inc.",Released
8,Jun 13 2022 10:29AM,239442,15,PNC356613A,"Person & Covey, Inc.",Released
9,Jun 13 2022 10:29AM,239442,15,PNC356621A,"Person & Covey, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
39,239442,Released,34
40,239445,Released,1
41,239447,Released,3
42,239448,Released,1
43,239450,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239442,NaN,NaN,34.0
239445,NaN,NaN,1.0
239447,NaN,NaN,3.0
239448,NaN,NaN,1.0
239450,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239387,12.0,8.0,1.0
239388,0.0,0.0,1.0
239389,0.0,0.0,1.0
239391,4.0,10.0,1.0
239392,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239387,12,8,1
239388,0,0,1
239389,0,0,1
239391,4,10,1
239392,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239387,12,8,1
1,239388,0,0,1
2,239389,0,0,1
3,239391,4,10,1
4,239392,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239387,12,8,1
1,239388,,,1
2,239389,,,1
3,239391,4,10,1
4,239392,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 13 2022 10:35AM,239450,15,"Virtus Pharmaceuticals, LLC"
1,Jun 13 2022 10:34AM,239448,15,"Carwin Pharmaceutical Associates, LLC"
2,Jun 13 2022 10:34AM,239447,15,"Mizner Bioscience, LLC"
5,Jun 13 2022 10:30AM,239445,10,Beach Products Inc
6,Jun 13 2022 10:29AM,239442,15,"Person & Covey, Inc."
40,Jun 13 2022 10:24AM,239440,10,ISDIN Corporation
48,Jun 13 2022 10:24AM,239441,15,"Laser Pharmaceuticals, LLC"
55,Jun 13 2022 10:18AM,239438,10,ISDIN Corporation
102,Jun 13 2022 10:13AM,239433,15,"Brookfield Pharmaceuticals, LLC"
165,Jun 13 2022 10:12AM,239428,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 13 2022 10:35AM,239450,15,"Virtus Pharmaceuticals, LLC",,,1
1,Jun 13 2022 10:34AM,239448,15,"Carwin Pharmaceutical Associates, LLC",,,1
2,Jun 13 2022 10:34AM,239447,15,"Mizner Bioscience, LLC",,,3
3,Jun 13 2022 10:30AM,239445,10,Beach Products Inc,,,1
4,Jun 13 2022 10:29AM,239442,15,"Person & Covey, Inc.",,,34
5,Jun 13 2022 10:24AM,239440,10,ISDIN Corporation,,,8
6,Jun 13 2022 10:24AM,239441,15,"Laser Pharmaceuticals, LLC",,,7
7,Jun 13 2022 10:18AM,239438,10,ISDIN Corporation,,,47
8,Jun 13 2022 10:13AM,239433,15,"Brookfield Pharmaceuticals, LLC",,,63
9,Jun 13 2022 10:12AM,239428,10,ISDIN Corporation,,,40


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 13 2022 10:35AM,239450,15,"Virtus Pharmaceuticals, LLC",1,,
1,Jun 13 2022 10:34AM,239448,15,"Carwin Pharmaceutical Associates, LLC",1,,
2,Jun 13 2022 10:34AM,239447,15,"Mizner Bioscience, LLC",3,,
3,Jun 13 2022 10:30AM,239445,10,Beach Products Inc,1,,
4,Jun 13 2022 10:29AM,239442,15,"Person & Covey, Inc.",34,,
5,Jun 13 2022 10:24AM,239440,10,ISDIN Corporation,8,,
6,Jun 13 2022 10:24AM,239441,15,"Laser Pharmaceuticals, LLC",7,,
7,Jun 13 2022 10:18AM,239438,10,ISDIN Corporation,47,,
8,Jun 13 2022 10:13AM,239433,15,"Brookfield Pharmaceuticals, LLC",63,,
9,Jun 13 2022 10:12AM,239428,10,ISDIN Corporation,40,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 13 2022 10:35AM,239450,15,"Virtus Pharmaceuticals, LLC",1,,
1,Jun 13 2022 10:34AM,239448,15,"Carwin Pharmaceutical Associates, LLC",1,,
2,Jun 13 2022 10:34AM,239447,15,"Mizner Bioscience, LLC",3,,
3,Jun 13 2022 10:30AM,239445,10,Beach Products Inc,1,,
4,Jun 13 2022 10:29AM,239442,15,"Person & Covey, Inc.",34,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 13 2022 10:35AM,239450,15,"Virtus Pharmaceuticals, LLC",1.0,NaN,NaN
1,Jun 13 2022 10:34AM,239448,15,"Carwin Pharmaceutical Associates, LLC",1.0,NaN,NaN
2,Jun 13 2022 10:34AM,239447,15,"Mizner Bioscience, LLC",3.0,NaN,NaN
3,Jun 13 2022 10:30AM,239445,10,Beach Products Inc,1.0,NaN,NaN
4,Jun 13 2022 10:29AM,239442,15,"Person & Covey, Inc.",34.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 13 2022 10:35AM,239450,15,"Virtus Pharmaceuticals, LLC",1.0,0.0,0.0
1,Jun 13 2022 10:34AM,239448,15,"Carwin Pharmaceutical Associates, LLC",1.0,0.0,0.0
2,Jun 13 2022 10:34AM,239447,15,"Mizner Bioscience, LLC",3.0,0.0,0.0
3,Jun 13 2022 10:30AM,239445,10,Beach Products Inc,1.0,0.0,0.0
4,Jun 13 2022 10:29AM,239442,15,"Person & Covey, Inc.",34.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3591337,383.0,27.0,4.0
12,239425,27.0,7.0,0.0
15,1676093,132.0,0.0,0.0
16,239407,0.0,1.0,0.0
18,1436432,6.0,0.0,0.0
19,718236,3.0,8.0,12.0
20,478808,15.0,0.0,0.0
22,478777,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3591337,383.0,27.0,4.0
1,12,239425,27.0,7.0,0.0
2,15,1676093,132.0,0.0,0.0
3,16,239407,0.0,1.0,0.0
4,18,1436432,6.0,0.0,0.0
5,19,718236,3.0,8.0,12.0
6,20,478808,15.0,0.0,0.0
7,22,478777,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,383.0,27.0,4.0
1,12,27.0,7.0,0.0
2,15,132.0,0.0,0.0
3,16,0.0,1.0,0.0
4,18,6.0,0.0,0.0
5,19,3.0,8.0,12.0
6,20,15.0,0.0,0.0
7,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,383.0
1,12,Released,27.0
2,15,Released,132.0
3,16,Released,0.0
4,18,Released,6.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,22
Status,,,,,,,,
Completed,4.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0
Executing,27.0,7.0,0.0,1.0,0.0,8.0,0.0,0.0
Released,383.0,27.0,132.0,0.0,6.0,3.0,15.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,22
0,Completed,4.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0
1,Executing,27.0,7.0,0.0,1.0,0.0,8.0,0.0,0.0
2,Released,383.0,27.0,132.0,0.0,6.0,3.0,15.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,22
0,Completed,4.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0
1,Executing,27.0,7.0,0.0,1.0,0.0,8.0,0.0,0.0
2,Released,383.0,27.0,132.0,0.0,6.0,3.0,15.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()